In [ ]:
from src.download_raw import download_raw
from src.split_timeseries import split_timeseries
from src.split_to_patches import split_to_patches
from src.create_bactsam_dataset import create_bactsam_dataset
from src.train_bactsam_model import train_bactsam_model
from src.run_inference import run_inference

# Step one
We need to first download our raw OME-TIFF image stack into the `./raw` folder. The name of the image will be `raw_bact.tif`.

In [ ]:
url = "" # CHANGE ME
download_raw(url)

# Step two
We need to extract each time step as its own OME-TIFF image. Each image will be saved into the `./raw/time_steps` folder and will have a name based on this template: `bact_sub_000.tif`. There should be in total 100 images after this operation.

In [ ]:
split_timeseries()

# Step three
We need to split each time step image and ground truth mask into 64 patches of 256x256 pixels as the original image has a size of 2048x2048. The ground truth masks are located in the `./ground_truth/masks` folder.

In [ ]:
split_to_patches(12, 1)

# Step four
After splitting the images and masks into patches, a dataset is created for training and saved to disk into the `./bact_dataset` folder.

In [ ]:
create_bactsam_dataset()

# Step five
Now we can train our model.

In [ ]:
train_bactsam_model(num_epochs=10, batch_size=15)

# Step 6
Run inference with the trained model, which is saved at `./model`. Will write output masks to `./outputs`.

In [ ]:
from datasets import Dataset
raw_dataset = Dataset.load_from_disk("./bact_dataset/validation") # type: ignore
images = [raw_dataset[i]["image"] for i in range(0,4*64,4)] # need to select every 4th image because we artificially increased the number of images by 4
run_inference(images)